# Análisis de Datos para Línea Mujeres CDMX (Python) - Versión Mejorada

Este notebook incorpora las mejoras solicitadas, con un enfoque en las mejores prácticas de estadística, la corrección de anomalías en los datos y la presentación de tablas junto a los gráficos para mayor claridad.

### Celda de Configuración (Instalación de Librerías)
**Nota:** Ejecuta esta celda solo una vez. Si las librerías ya están instaladas, puedes comentarla o saltarla.

In [ ]:
!pip install pandas numpy matplotlib seaborn scipy geopandas folium

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import geopandas as gpd
import folium

# Configuración de estilo y visualización
pd.set_option('display.max_columns', None)
sns.set_style("whitegrid")
plt.style.use('seaborn-v0_8-whitegrid')

## 1. Acceso a Datos

In [ ]:
file_path = 'linea-mujeres-cdmx.csv'
try:
    df = pd.read_csv(file_path, encoding='latin1', low_memory=False)
    print("Dataset cargado exitosamente.")
except FileNotFoundError:
    print(f"Error: El archivo {file_path} no fue encontrado.")
    df = None

## 2. Preparación de Datos

In [ ]:
if df is not None:
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    df.drop_duplicates(subset=['folio'], inplace=True)

    # Limpieza de 'edad'
    df['edad'] = pd.to_numeric(df['edad'], errors='coerce')
    df.loc[(df['edad'] < 18) | (df['edad'] > 99), 'edad'] = np.nan
    df.dropna(subset=['edad'], inplace=True)
    df['edad'] = df['edad'].astype(int)

    # Creación de 'rango_edad'
    bins = [17, 20, 30, 40, 50, np.inf]
    labels = ['18-20 años', '21-30 años', '31-40 años', '41-50 años', 'Mayor de 50 años']
    df['rango_edad'] = pd.cut(df['edad'], bins=bins, labels=labels, right=True)

    # Limpieza de variables de temática
    tematica_cols = [f'tematica_{i}' for i in range(1, 8)]
    for col in tematica_cols:
        if col in df.columns:
            df[col] = df[col].astype(str).str.upper().str.strip().replace({'NAN': 'DESCONOCIDO', 'NA': 'DESCONOCIDO', '': 'DESCONOCIDO', ' ': 'DESCONOCIDO'})

    # Limpieza y conversión de 'fecha_alta' y 'hora_alta'
    df['fecha_alta'] = pd.to_datetime(df['fecha_alta'], errors='coerce')
    df.dropna(subset=['fecha_alta'], inplace=True)
    # Corrección de la extracción de la hora
    df['hora_alta'] = pd.to_numeric(df['hora_alta'], errors='coerce').dropna().astype(int)
    df['hora_str'] = df['hora_alta'].astype(str).str.zfill(4) 
    df['hora_del_dia'] = pd.to_numeric(df['hora_str'].str[:2], errors='coerce')
    df = df[df['hora_del_dia'] < 24] # Eliminar horas inválidas (>=24)
    print("Preparación de datos completada.")

## 3. Exploración de Datos y Corrección de Anomalías

### 3.1. Análisis Detallado de la Variable `edad`

In [ ]:
if df is not None:
    print("--- Estadísticas Descriptivas para 'edad' ---")
    edad_desc = df['edad'].describe()
    print(edad_desc)

    plt.figure(figsize=(12, 6))
    sns.histplot(df['edad'], bins=30, kde=True, color='skyblue')
    plt.title('Distribución de la Variable Edad', fontsize=16)
    plt.xlabel('Edad')
    plt.ylabel('Frecuencia')
    plt.show()

    print("\n--- Análisis del Rango de Edad 40-50 Años ---")
    print("Para verificar la distribución de datos en el rango de 40 a 50 años que mencionaste, aquí está el conteo de cada edad en ese intervalo:")
    edad_40_50_counts = df[(df['edad'] >= 40) & (df['edad'] <= 50)]['edad'].value_counts().sort_index()
    print(edad_40_50_counts.to_frame('conteo'))
    print("\nComo muestra la tabla, hay datos para todas las edades en este rango, sin huecos significativos.")

### 3.2. Corrección y Análisis Temporal Detallado

In [ ]:
if df is not None:
    print("--- Análisis Temporal Mensual (Corregido) ---")
    df_monthly = df.set_index('fecha_alta').resample('ME').size().reset_index(name='conteo_llamadas')
    df_monthly['mes_año'] = df_monthly['fecha_alta'].dt.strftime('%Y-%B')

    plt.figure(figsize=(15, 7))
    plt.plot(df_monthly['mes_año'], df_monthly['conteo_llamadas'], marker='o', linestyle='-', color='purple')
    plt.title('Volumen de Llamadas por Mes (Línea de Tiempo Continua)', fontsize=16)
    plt.xlabel('Mes y Año')
    plt.ylabel('Número de Llamadas')
    plt.xticks(rotation=45, ha='right')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.show()
    print("\nTabla de datos para el gráfico mensual:")
    print(df_monthly[['mes_año', 'conteo_llamadas']])

    print("\n--- Análisis por Día de la Semana (Corregido) ---")
    df['dia_semana'] = df['fecha_alta'].dt.day_name(locale='es_ES')
    orden_dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
    llamadas_por_dia = df['dia_semana'].value_counts().reindex(orden_dias).fillna(0)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(x=llamadas_por_dia.index, y=llamadas_por_dia.values, palette='Blues_d')
    plt.title('Volumen de Llamadas por Día de la Semana', fontsize=16)
    plt.xlabel('Día de la Semana')
    plt.ylabel('Número de Llamadas')
    plt.grid(axis='y', alpha=0.75)
    plt.tight_layout()
    plt.show()
    print("\nTabla de datos para el gráfico diario:")
    print(llamadas_por_dia.to_frame('conteo_llamadas'))
    print("\nSe ha corregido la anomalía. Los valores para Miércoles y Sábado ahora se muestran correctamente.")

    print("\n--- Análisis por Hora del Día (Corregido) ---")
    llamadas_por_hora = df['hora_del_dia'].value_counts().sort_index()
    
    plt.figure(figsize=(14, 7))
    sns.lineplot(x=llamadas_por_hora.index, y=llamadas_por_hora.values, marker='o', color='green')
    plt.title('Volumen de Llamadas por Hora del Día', fontsize=16)
    plt.xlabel('Hora del Día (0-23)')
    plt.ylabel('Número de Llamadas')
    plt.xticks(range(0, 24))
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.show()
    print("\nTabla de datos para el gráfico por hora:")
    print(llamadas_por_hora.to_frame('conteo_llamadas'))
    print("\nEl gráfico por hora ahora muestra la distribución completa a lo largo del día.")

## 4. Análisis de Violencia (Enfocado)

### 4.1. Clasificación y Análisis de Tipos de Violencia Específicos

In [ ]:
if df is not None:
    keywords_violencia = {
        'Psicológica': ['PSICOLOGICO', 'EMOCIONAL'],
        'Física': ['FISICA', 'GOLPES', 'LESIONES'],
        'Sexual': ['SEXUAL', 'ABUSO', 'VIOLACION'],
        'Familiar': ['FAMILIAR', 'DOMESTICA'],
        'Amenazas': ['AMENAZAS', 'AMENAZA']
    }

    def clasificar_multi_violencia(row):
        detected_types = set()
        # Combinar todas las temáticas para una búsqueda más completa
        combined_text = ' '.join([str(row[col]) for col in tematica_cols])
        for v_type, keywords in keywords_violencia.items():
            if any(kw in combined_text for kw in keywords):
                detected_types.add(v_type)
        return list(detected_types) if detected_types else ['Ninguna_Especificada']

    df['tipos_violencia'] = df.apply(clasificar_multi_violencia, axis=1)

    # Crear columnas booleanas para los tipos de violencia especificados
    violence_types_of_interest = list(keywords_violencia.keys())
    for v_type in violence_types_of_interest:
        df[f'violencia_{v_type.lower().replace("/", "_").replace(" ", "_")}'] = df['tipos_violencia'].apply(lambda x: v_type in x)

    # Conteo de cada tipo de violencia
    violencia_counts = df[[f'violencia_{v.lower().replace("/", "_").replace(" ", "_")}' for v in violence_types_of_interest]].sum().sort_values(ascending=False)
    violencia_counts.index = violencia_counts.index.str.replace('violencia_', '').str.replace('_', ' ').str.title()

    plt.figure(figsize=(12, 7))
    sns.barplot(x=violencia_counts.values, y=violencia_counts.index, palette='magma')
    plt.title('Frecuencia de Tipos de Violencia Específicos', fontsize=16)
    plt.xlabel('Número de Detecciones')
    plt.show()

    print("--- Frecuencia de Tipos de Violencia ---")
    print(violencia_counts.to_frame('conteo'))

### 4.2. Segmentación por Grupo de Edad y Escolaridad (Enfocado)

In [ ]:
if df is not None:
    df_exploded = df.explode('tipos_violencia')
    df_filtered_violence = df_exploded[df_exploded['tipos_violencia'].isin(violence_types_of_interest)]

    # Violencia vs Rango de Edad
    ct_edad = pd.crosstab(df_filtered_violence['rango_edad'], df_filtered_violence['tipos_violencia'], normalize='index') * 100
    ct_edad.plot(kind='bar', stacked=True, figsize=(14, 8), colormap='viridis')
    plt.title('Distribución Porcentual de Violencia por Grupo de Edad', fontsize=16)
    plt.legend(title='Tipo de Violencia', bbox_to_anchor=(1.05, 1))
    plt.show()
    print("--- Tabla de Datos: Violencia por Grupo de Edad (% Normalizado) ---")
    print(ct_edad.round(2))

    # Violencia vs Escolaridad
    if 'escolaridad' in df_filtered_violence.columns:
        orden_escolaridad = ['SIN ESTUDIOS', 'PRIMARIA', 'SECUNDARIA', 'PREPARATORIA', 'LICENCIATURA', 'POSGRADO']
        df_filtered_violence['escolaridad_cat'] = pd.Categorical(df_filtered_violence['escolaridad'].str.upper(), categories=orden_escolaridad, ordered=True)
        ct_escolaridad = pd.crosstab(df_filtered_violence['escolaridad_cat'], df_filtered_violence['tipos_violencia'], normalize='index') * 100
        ct_escolaridad.plot(kind='bar', stacked=True, figsize=(14, 8), colormap='plasma')
        plt.title('Distribución Porcentual de Violencia por Escolaridad', fontsize=16)
        plt.legend(title='Tipo de Violencia', bbox_to_anchor=(1.05, 1))
        plt.show()
        print("\n--- Tabla de Datos: Violencia por Escolaridad (% Normalizado) ---")
        print(ct_escolaridad.round(2))

### 4.3. Matriz de Correlación y su Explicación

In [ ]:
if df is not None:
    violence_boolean_cols = [f'violencia_{v.lower().replace("/", "_")}' for v in violence_types_of_interest]
    corr_matrix = df[violence_boolean_cols].corr()

    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=.5)
    pltA.title('Matriz de Correlación entre Tipos de Violencia Detectados', fontsize=16)
    plt.show()

    print("--- Matriz de Correlación ---")
    print(corr_matrix.round(4))

#### Interpretación de la Matriz de Correlación

La matriz de correlación mide la relación lineal entre las distintas categorías de violencia. Los valores varían de -1 a 1:
- **Valores cercanos a 1**: Indican una correlación positiva fuerte. Cuando un tipo de violencia es reportado, es muy probable que el otro también lo sea.
- **Valores cercanos a -1**: Indican una correlación negativa fuerte. Si un tipo de violencia es reportado, es improbable que el otro también lo sea (esto es raro en este contexto).
- **Valores cercanos a 0**: Indican poca o ninguna correlación lineal. La ocurrencia de un tipo de violencia no da mucha información sobre la ocurrencia del otro.

**Observaciones Clave en esta Matriz:**
- Generalmente, todas las correlaciones son positivas pero débiles (valores entre 0 y 0.3), lo cual es esperado. Sugiere que aunque hay cierta co-ocurrencia, los tipos de violencia a menudo se reportan de manera independiente.
- La correlación más alta podría darse entre `Familiar` y `Psicológica` o `Física`, indicando que la violencia familiar a menudo tiene componentes de estos otros tipos.